Replication of Civil Wars Analyses for Yager Working Paper:
The Psychological Effects of Ethnic Targeting on Civil War

In [16]:
# Loading packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sklearn
import os 
import statsmodels.api as sm
import statsmodels.formula.api as smf


Setting working directory: 

In [17]:
# Set WD
working_directory = "C:\\Users\\nyager\\Desktop\\YagerMLCode\\MachineLearning_Fall24\\CivilWars_Analyses"
os.chdir(working_directory)

# Confirm the change
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\nyager\Desktop\YagerMLCode\MachineLearning_Fall24\CivilWars_Analyses


Load data: 

In [18]:
# Loading data
data = pd.read_csv("rawdata/data_groupyear.csv")

# Display the first few rows of the dataset
print(data.head())

   gwgroupid  year  countries_gwid onset_id    countryname  territorial_c  \
0     201000  2009               2      NaN  United States            NaN   
1     201000  2010               2      NaN  United States            NaN   
2     201000  2011               2      NaN  United States            NaN   
3     201000  2012               2      NaN  United States            NaN   
4     201000  2013               2      NaN  United States            NaN   

   intensity_level  active_dyads  active_dyads_change  ongoing_any  ...  \
0                0             0                  0.0            0  ...   
1                0             0                  0.0            0  ...   
2                0             0                  0.0            0  ...   
3                0             0                  0.0            0  ...   
4                0             0                  0.0            0  ...   

   l_yrs_excl  l_yrs_disc  pys_low2  pys_low3  wys_low2  wys_low3  wys_high2  \
0     

C:\Users\nyager\AppData\Local\Temp\ipykernel_25356\380121620.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("rawdata/data_groupyear.csv")


First analysis: Markov Model Replication

In [23]:
# Is the data sorted by group and time? 
data = data.sort_values(by=['gwgroupid', 'year'])

# Lagged variable generation (for analysis)
data['intensity_level_lag'] = data.groupby('gwgroupid')['intensity_level'].shift(1)

# Generating interaction terms
data['intensity_1_interaction'] = data['Target_Gov_l1'] * data['int_level1_lag']
data['intensity_2_interaction'] = data['Target_Gov_l1'] * data['int_level2_lag']


# Calculate transition matrix (basic example using pandas crosstab)
transition_matrix = pd.crosstab(data['intensity_level_lag'], data['intensity_level'], normalize='index')
print("Transition Matrix:\n", transition_matrix)

# Multinomial logistic regression (close to logit in Stata)
formula = 'intensity_level ~ intensity_1_interaction + intensity_2_interaction + peaceyrs_lowint + waryrs + ln_gdppc_l + ln_pop_l + family_warhist + cincidence_flag_l + status_excl'
mlogit_model = smf.mnlogit(formula, data=data)
mlogit_results = mlogit_model.fit()

# Displaying results
print(mlogit_results.summary())


Transition Matrix:
 intensity_level             0         1         2
intensity_level_lag                              
0.0                  0.988162  0.010967  0.000872
1.0                  0.285199  0.648014  0.066787
2.0                  0.083770  0.214660  0.701571
Optimization terminated successfully.
         Current function value: 0.104535
         Iterations 11
                          MNLogit Regression Results                          
Dep. Variable:        intensity_level   No. Observations:                14527
Model:                        MNLogit   Df Residuals:                    14507
Method:                           MLE   Df Model:                           18
Date:                Wed, 06 Nov 2024   Pseudo R-squ.:                  0.5433
Time:                        09:33:48   Log-Likelihood:                -1518.6
converged:                       True   LL-Null:                       -3325.0
Covariance Type:            nonrobust   LLR p-value:                     0